In [155]:
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

In [156]:
train_df, test_df = train_test_split(pd.read_csv('data/movie_db.csv'))

In [182]:
def count_genre(genre, genre2idx):
    
    genre_counts = {genre : 0 for genre in genre2idx.keys()}
    
    for i in genre:
        try:
            genres = i.split(',')

            for g in genres:

                g = g.strip()

                genre_counts[g] += 1
                
        except:
            None
            
    return genre_counts

def get_genres(genre):
    genre_set = []
    num=0
    failed=[]
    for i in genre:
        try:
            genres = i.split(',')

            for g in genres:

                g = g.strip()

                if g not in genre_set:
                    genre_set.append(g)
        except:
            failed.append(num)
        num+=1
        
    idx2genre = dict(enumerate(genre_set))
    genre2idx = {g : idx for idx, g in idx2genre.items()}
    
    return idx2genre, genre2idx, failed

def encode_genres(genres, genre2idx):
    
    encoded_genres = []
    failed=[]
    vector_size = len(genre2idx)
    num = 0
    for i in genres:
        try:
            empty_vec = np.zeros(vector_size)    
            encoded = [genre2idx[x.strip()] for x in i.split(',')]

            for i in encoded:
                empty_vec[i] = 1
        except:
            failed.append(num)
            continue
        num+=1
        encoded_genres.append(empty_vec)
        
    return torch.Tensor(encoded_genres), failed

def encode_genre(genre, genre2idx):
    
    encoded_genre = torch.LongTensor([genre2idx[g.strip()] for g in genre])
        
    return torch.encoded_genre

In [183]:
genre = train_df['genre'].tolist()
test_genre = test_df['genre'].tolist()
idx2genre, genre2idx, failed = get_genres(genre)
genre_counts = count_genre(genre, genre2idx)
encoded_genres,failed = encode_genres(genre, genre2idx)
test_encoded_genres, test_failed = encode_genres(test_genre, genre2idx)

In [184]:
train_df.loc[0]

id                                                             1
title                                                  Toy Story
genre              Animation, Adventure, Comedy, Family, Fantasy
imdb_link                                              tt0114709
plot           A little boy named Andy loves to be in his roo...
poster_path                        data/posters/1-toy-story.jpeg
Name: 0, dtype: object

In [185]:
DATASET_PATH = 'data/posters/'
train_posters = train_df['poster_path'].tolist()

In [186]:
process =  transforms.Compose([transforms.Resize(255), 
                                       transforms.CenterCrop(224),  
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(), 
                                       #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                              ])


In [187]:
from PIL import Image

In [188]:
a = Image.open(train_posters[0])

In [198]:
class PosterDataset(Dataset):
    
    def __init__(self, df, encoded_genres, transform):
        
        self.df = df
        self.genre2idx = encoded_genres
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        x = self.transform(Image.open(self.df.iloc[idx]['poster_path']))
        y = encoded_genres[idx]
        
        return {
            'image' : x,
            'genre' : y
        }

In [203]:
train_dataset = PosterDataset(train_df, genre2idx, process)
test_dataset = PosterDataset(test_df, genre2idx, process)

In [338]:
train_dataloader = DataLoader(train_dataset, shuffle=True)
test_dataloader =DataLoader(test_dataset)

## CNN Model


In [339]:
import torch.nn as nn
import torch.functional as F

In [428]:
class CNN(nn.Module):
    
    def __init__(self, out_features):
        super(CNN,self).__init__()
        
        self.out_features = out_features
        
        self.layer1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3)
        self.layer2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3)
        
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.activation = nn.ReLU()
        
        self.linlayer1 = nn.Linear(13824, 300)
        self.linlayer2 = nn.Linear(300, 60)
        self.outputlayer = nn.Linear(60, self.out_features)
        
        self.softmax = nn.Sigmoid()
        
    def forward(self, input):
        
        out = self.layer1(input)
        out = self.activation(out)
        out = self.max_pool(out)
        
        #print(out.shape)
        
        out = self.layer2(out)
        out = self.activation(out)
        out = self.max_pool(out)
        
        #print(out.shape)
        
        #out = out.reshape(-1, 24 *4 *4)
        out = out.view(out.size(0),-1)
        
        #print(out.shape)
        
        out = self.linlayer1(out)
        out = self.activation(out)
        
        #print(out.shape)
        
        out = self.linlayer2(out)
        out = self.activation(out)
        
        out = self.outputlayer(out)
        
        return out

In [429]:
model = CNN(len(genre2idx
               ))

In [430]:
output_size = len(genre2idx)
n_layers = 1
lr = 0.001
device = 'cpu'

In [431]:
param_lrs = [{'params': param, 'lr': lr} for param in model.parameters()]
optimizer = torch.optim.Adam(param_lrs, lr=lr)
criterion = nn.BCEWithLogitsLoss().to(device)
NUM_EPOCHS=10

In [435]:
from tqdm.notebook import tqdm

In [438]:
 
for epoch in tqdm(range(0, NUM_EPOCHS), total=NUM_EPOCHS):

    model.train()
    avg_loss = 0
    failed = []
    
    for i in tqdm(train_dataloader):

        image = i['image']
        out = model(image)

        loss= criterion(out, i['genre'])

        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_dataloader)

    print(avg_loss)
    

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/374 [00:00<?, ?it/s]

0.33019700068681623


  0%|          | 0/374 [00:00<?, ?it/s]

0.32481925783150994


...

0.31688270100297816


  0%|          | 0/374 [00:00<?, ?it/s]

0.3165243206655273


  0%|          | 0/374 [00:00<?, ?it/s]

0.3183284872834059


In [420]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def get_index(preds):
    empty = []
    
    for i in preds:
        if i > 0.5:
            empty.append(1)
            
        else:
            empty.append(0)
            
    return empty
            

In [422]:
sigmoid(out.detach().numpy())

array([[0.52721506, 0.50683796, 0.49765334, 0.5051012 , 0.47298482,
        0.5161388 , 0.5269805 , 0.515773  , 0.49353278, 0.50992537,
        0.52013505, 0.47583356, 0.4751174 , 0.47212914, 0.4819235 ,
        0.4986639 , 0.48364657, 0.48591143, 0.5316629 , 0.48052526,
        0.520928  ]], dtype=float32)

In [433]:
preds = []
set_true = []
true=[]
model.eval()

num = 0
for i in tqdm(test_dataloader):

    x = i['image']
    y = i['genre']
    out = model(x)
    pred = sigmoid(out.detach().numpy())
    preds.append(get_index(pred[0]))
    set_true.append((i['genre'],i['title']))
    num += 1
    true.append(y.squeeze(0).cpu().numpy())

tensor(0.7017, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)